<a href="https://colab.research.google.com/github/sejal-godbole/Genai/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Libraries

In [ ]:
pip install transformers datasets torch accelerate


Upload Your Story Dataset

In [ ]:
%%writefile stories.txt
Once upon a time, a girl found a magical book in the forest.

The boy followed the light and entered a secret cave.

In a small village, everyone feared the old tree.

A lonely robot learned what friendship meant.

The princess escaped the tower using her intelligence.


Writing stories.txt


In [ ]:
from google.colab import files
files.upload()


Load GPT-2 Model & Tokenizer

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Load model
model = GPT2LMHeadModel.from_pretrained("gpt2")

print("Model loaded successfully!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded successfully!


Load Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("text", data_files={"train": "stories.txt"})

print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9
    })
})


Tokenization

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenizer.pad_token = tokenizer.eos_token
tokenized_data = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Data Collator

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


Training Settings

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2_story_model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    fp16=True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    data_collator=data_collator,
)

trainer.train()


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=15, training_loss=4.535655721028646, metrics={'train_runtime': 15.7759, 'train_samples_per_second': 1.711, 'train_steps_per_second': 0.951, 'total_flos': 1763721216000.0, 'train_loss': 4.535655721028646, 'epoch': 3.0})

Save Model

In [ ]:
model.save_pretrained("./my_story_model")
tokenizer.save_pretrained("./my_story_model")

print("Model Saved!")


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Model Saved!


Generate New Stories

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("./my_story_model")
model = GPT2LMHeadModel.from_pretrained("./my_story_model")

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

prompt = "Once upon a time"

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    padding=True,
    truncation=True
)

input_ids = inputs["input_ids"].to(model.device)
attention_mask = inputs["attention_mask"].to(model.device)

output = model.generate(
    input_ids,
    max_length=200,
    temperature=0.8,
    top_k=50,
    top_p=0.9,
    do_sample=True
)

story = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Story:\n")
print(story)


Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Story:

Once upon a time, when it seemed the world was drifting in circles, the sun was shining in a manner of light that no man ever imagined. But now, when it was only a fraction of a second into the night, the sun shone in the heavens and the earth and the stars shone in the clouds and the skies and the sky was shining in the clouds.

The ancient Gods of the sky began to worship the sun, and the sun shone in the heavens and the earth and the stars shone in the clouds and the skies and the sky was shining in the clouds.

The gods of the sky, like the sun, were beginning to worship the sun, and the sun shone in the heavens and the earth and the stars shone in the clouds and the skies and the sky was shining in the clouds.

The gods of the sky, like the sun, were beginning to worship the sun, and the sun shone in the heavens and the earth and the stars shone in the
